In [1]:
from ipycanvas import Canvas, hold_canvas
import numpy as np
from matplotlib.colors import rgb2hex, Normalize
import matplotlib
import svg 

In [2]:
x = []
y = []
r = []

def add_circle(x1, y1, r1):
    x.append(x1)
    y.append(y1)
    r.append(r1)
    
def reflect_circle(z1, z2, z3, z4, k1, k2, k3, k4):
    """ Reflects the circle with center z4 and radius k4 to the second solution of the Descartes circle theorem."""
 
    k = 2*(k1+k2+k3) - k4
    z = (2*(k1*z1 + k2*z2 + k3*z3) - k4*z4) / k
    
    return z, k

def gasket(z1,z2,z3,z4,k1,k2,k3,k4,level=0,maxlevel=1,k_max=800):
    """ Recursive function to generate the Apollonian gasket."""
       
    # At the first level, four circles are added,
    # while at the next levels only the unique reflected
    # circle from the recursion is added.
    
    # The recursion adds three circles.
    if level == 0:
        # Add the first four kissing circles
        add_circle(z1.real, z1.imag, 1/np.abs(k1))
        add_circle(z2.real, z2.imag, 1/np.abs(k2))
        add_circle(z3.real, z3.imag, 1/np.abs(k3))
        add_circle(z4.real, z4.imag, 1/np.abs(k4))
                
    else:
        # add only the generated reflected circle
        add_circle(z4.real, z4.imag, 1/np.abs(k4))
    
    # Recursion.
    # Add 4 circles at level 0, and 3 circles at each level after that
    if level < maxlevel:
        if level == 0:
            z, k = reflect_circle(z1, z2, z3, z4, k1, k2, k3, k4)    
            gasket(z1, z2, z3, z, k1, k2, k3, k, level=level+1,maxlevel=maxlevel)
        
        z, k = reflect_circle(z1, z2, z4, z3, k1, k2, k4, k3)

        # Continue to next level only if the radius of the new circle is smaller than the minimum radius
        if k < k_max:  
            gasket(z1, z2, z4, z, k1, k2, k4, k, level=level+1,maxlevel=maxlevel)
        
        z, k = reflect_circle(z1, z4, z3, z2, k1, k4, k3, k2) 
           
        if k < k_max:
            gasket(z1, z4, z3, z, k1, k4, k3, k, level=level+1,maxlevel=maxlevel)
        
        z, k = reflect_circle(z4, z2, z3, z1, k4, k2, k3, k1)    
    
        if k < k_max:
            gasket(z4, z2, z3, z, k4, k2, k3, k, level=level+1,maxlevel=maxlevel)


In [3]:
# Seed circles
z1 = -1.0
z2 = 1.0
z3 = 0.0

k1 = 1.0
k2 = 1.0
k3 = -0.5
k4 = 1.5
z4 = 4j/3

# Generate the gasket
gasket(z1, z2, z3, z4, k1, k2, k3, k4, maxlevel=25)
x, y, r = np.array(x), np.array(y), np.array(r)
k = 1/r
 

In [5]:

# set up viewing window
x_min = -2
x_max = 2
y_min = -2
y_max = 2

# Size of the figure in points
pix = 1000

def canvas_coords(x, y, r):
    """ Convert logical coordinates to canvas coordinates."""
    x = (np.array(x) - x_min) / (x_max - x_min) * pix
    y = pix - (np.array(y) - y_min) / (y_max - y_min) * pix
    r = np.array(r) / (x_max - x_min) * pix
    return x, y, r


# Convert logical coordinates to canvas coordinates
xx, yy, rr = canvas_coords(x, y, r)
# Find maximum curvature (smallest circle)
k_max = np.max(k)

# Choose color map
cmap = matplotlib.colormaps.get_cmap('viridis')

# Generate SVG elements
elements = []
for x1,y1,r1,k1 in zip(xx, yy, rr, k):
    index0 = 0.0005 * k1**.5
    color = cmap(255*index0)
    fill_style = rgb2hex(color)
    stroke_style = rgb2hex((0,0,0))
    elements.append(svg.Circle(cx=x1, cy=y1, r=r1, fill=fill_style, stroke=stroke_style, stroke_width=.25))



In [6]:
# Create SVG canvas
canvas = svg.SVG(
    width=pix,
    height=pix,
    elements=elements
)

# Save SVG to file
with open('gasket.svg', 'w') as f:
    f.write(str(canvas))